# QMMM workflow using GROMACS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use methane as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://votca.github.io/xtp_map.html)
* [xtp_parallel](https://votca.github.io/xtp_parallel.html)
* [xtp_run](https://votca.github.io/xtp_run.html)
* [xtp_tools](https://votca.github.io/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          calculator options
  -t [ --nthreads ] arg (=1)    number of threads to create

Tools:
  -e [ --execute ] arg        List of tools separated by ',' or ' '
  -l [ --list ]               Lists all available tools
  -d [ --description ] arg    Short description of a tool
  -n [ --name ] arg           Name of the job to run



### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [2]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [3]:
!rm -f state.hdf5

Import an small python module to edit xml files

In [4]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file

runs the mapping from MD coordinates to segments and creates an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/). You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/).

In [5]:
!xtp_map -t MD_FILES/topol.tpr -c MD_FILES/conf.gro -s system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_map, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Reading file MD_FILES/topol.tpr, VERSION 5.1.1 (single precision)
Note: file tpx version 103, software tpx version 116
Frame with id 0 was not in statefile state.hdf5 ,adding it now.
Writing MD topology (step = 0, time = 0) to state.hdf5
... . 


##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. We will need to copy the input from the `VOTCA` installation path to our local copy.

In [6]:
!cp "$VOTCASHARE/xtp/xml/mapchecker.xml" OPTIONFILES/

The `${VOTCASHARE}` environmental variable is set to the path that you provided during the `VOTCA` [installation](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md), by the default is set to `/usr/local/votca`.

We need to update the `mapchecker` options with the file containing the definition of the system. For doing, so we will use the python module `xml_editor` that we have imported above. The first argument is the name of the calculator (i.e. `mapchecker`), the seconds argument is the name of the option to change and the third one, the value to use.

In [7]:
edit_calculator("mapchecker", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/mapchecker.xml' has been set to 'system.xml'


In the [mapchecker section of the manual](https://votca.github.io/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [8]:
!xtp_run -e mapchecker -o OPTIONFILES/mapchecker.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator
... mapchecker 
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... mapchecker 
Writing segments to md_segments_step_0.pdb
Writing qmmolecules to qm_segments_n_step_0.pdb
Writing polarsegments to mp_segments_e_step_0.pdb
Writing polarsegments to mp_segments_h_step_0.pdb

Changes have not been written to state file.


## Neighborlist Calculation

The following step is to determine the neighbouring pairs for exciton transport. We will need first to get a copy of the input 

In [9]:
cp "$VOTCASHARE/xtp/xml/neighborlist.xml" OPTIONFILES/

Then we will set the *cutoff* for the pairs to 0.5 nanometers. See the [neighborlist options](https://votca.github.io/neighborlist.html) for further information. 

In [10]:
exciton_cutoff = """
<exciton_cutoff>0.5</exciton_cutoff>
"""
segments = """
<segments>
  <type>DCV5T_ME3 DCV5T_ME3</type>
  <cutoff>3</cutoff>
</segments>
"""
add_section("neighborlist", exciton_cutoff)
add_section("neighborlist", segments)
edit_calculator("neighborlist", "constant", "0.6")

The option 'constant' on file 'OPTIONFILES/neighborlist.xml' has been set to '0.6'


Finally, we can run the calculation using 4 threads

In [11]:
!xtp_run -e neighborlist -o OPTIONFILES/neighborlist.xml -f state.hdf5 -t 4

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator
... neighborlist 
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... neighborlist  Using 4 threads
Evaluating 1000 segments for neighborlist. 
 ... ... Evaluating 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

 ... ... Created 21093 direct pairs.
 ... ... Determining classical pairs 
 ... ... Found 8586 classical pairs 

Writing MD topology (step = 0, time = 0) to state.hdf5
... . 


## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation

In [12]:
!cp "$VOTCASHARE/xtp/xml/einternal.xml" OPTIONFILES/
!xtp_run -e einternal -o OPTIONFILES/einternal.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator
... einternal 
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... einternal 
... ... Site, reorg. energies from system.xml.

... ... Read in site, reorg. energies for 1000 segments. 
Writing MD topology (step = 0, time = 0) to state.hdf5
... . 


## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file,

In [13]:
!cp qmmm_mm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "write"

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... qmmm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
... ... Writing job file qmmm_mm_jobs.xml
... ... In total 4001 jobs
Changes have not been written to state file.


The previous command generates a `qmmm_mm_jobs.xml` containing 4000 *MM* jobs to compute, if you examine that file, it should look something like

```xml
<jobs>
  <job>
    <id>0</id>
    <tag>Methane_0:n</tag>
    <input>
      <site_energies>0:n</site_energies>
      <regions>
        <region>
          <id>0</id>
          <segments>0:n</segments>
        </region>
      </regions>
    </input>
    <status>AVAILABLE</status>
  </job>    
...
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [14]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [15]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "run"
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "read"

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... qmmm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
MST ERR Job file = 'qmmm_mm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_mm_jobs.xml
MST ERR Registered 4000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ...  Regions created
T00 ERR ... Id: 0 type: polar size: 17 charge[e]= -9.29812e-16
T00 ERR ... Id: 1 type: static size: 254

## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [16]:
!cp "$VOTCASHARE/xtp/xml/eanalyze.xml" OPTIONFILES/
!xtp_run -e eanalyze -o OPTIONFILES/eanalyze.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator
... eanalyze 
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eanalyze 
... ... Short-listed 1000 segments (pattern='*')
... ... ... NOTE Statistics of site energies and spatial correlations thereof are based on the short-listed segments only. 
... ... ...      Statistics of site-energy differences operate on the full list.
... ... excited state e
... ... excited state h
... ... excited state s
... ... excited state t
Changes have not been written to state file.


You should now see a bunch of files prefix with `eanalyze` containing the histrogram and correlation functions.

In [17]:
!ls eanalyze*

eanalyze.pairhist_e.out  eanalyze.pairlist_s.out  eanalyze.sitehist_e.out
eanalyze.pairhist_h.out  eanalyze.pairlist_t.out  eanalyze.sitehist_h.out
eanalyze.pairhist_s.out  eanalyze.sitecorr_e.out  eanalyze.sitehist_s.out
eanalyze.pairhist_t.out  eanalyze.sitecorr_h.out  eanalyze.sitehist_t.out
eanalyze.pairlist_e.out  eanalyze.sitecorr_s.out
eanalyze.pairlist_h.out  eanalyze.sitecorr_t.out


## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We will first copy the input into our local folder

In [18]:
!cp "$VOTCASHARE/xtp/xml/eqm.xml" OPTIONFILES/

Now we set the GWBSE mode to `G0W0` and  the `ranges` to `full`. For more information, check the [eqm calculator options](https://votca.github.io/eqm.html).

In [19]:
edit_calculator("eqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("eqm", "gwbse_options.gwbse.mode", "G0W0")
edit_calculator("eqm", "map_file", "system.xml")

The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/eqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.mode' on file 'OPTIONFILES/eqm.xml' has been set to 'G0W0'
The option 'map_file' on file 'OPTIONFILES/eqm.xml' has been set to 'system.xml'


For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://votca.github.io/eqm.html).

In [20]:
edit_calculator("eqm", "gwbse_options.gwbse.tasks", "gw,singlets")

The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/eqm.xml' has been set to 'gw,singlets'


First we will write the job in a file and enable only the first 3,


In [21]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... eqm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eqm 
... ... Writing job file: eqm.jobs1000 jobs
Changes have not been written to state file.


Now, let run this 3 jobs

In [22]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j run -c 1

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... eqm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... eqm 
MST ERR Job file = 'eqm.jobs', cache size =  1
MST ERR Initialize jobs from eqm.jobs
MST ERR Registered 1000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2020-5-26 17:38:25 Evaluating site 0
T00 ERR ... Running DFT
T00 ERR ... Running GWBSE
T00 ERR ... Running ESPFIT
T00 ERR ... ===== Runnin

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

In [23]:
!cp "$VOTCASHARE/xtp/xml/iqm.xml" OPTIONFILES/

As in the previous section, we set the GWBSE mode to `G0W0`and the `ranges` to `full`, but we compute only the `gw` approximation. We are also going to compute a single excited state. For more information, check the [iqm calculator options](https://votca.github.io/iqm.html). We also want to compute the `singlet` couplings. 

In [24]:
edit_calculator("iqm", "states", "1")
edit_calculator("iqm", "map_file", "system.xml")
edit_calculator("iqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("iqm", "gwbse_options.gwbse.tasks", "gw")
edit_calculator("iqm", "bsecoupling.spin", "singlet")

The option 'states' on file 'OPTIONFILES/iqm.xml' has been set to '1'
The option 'map_file' on file 'OPTIONFILES/iqm.xml' has been set to 'system.xml'
The option 'gwbse_options.gwbse.ranges' on file 'OPTIONFILES/iqm.xml' has been set to 'full'
The option 'gwbse_options.gwbse.tasks' on file 'OPTIONFILES/iqm.xml' has been set to 'gw'
The option 'bsecoupling.spin' on file 'OPTIONFILES/iqm.xml' has been set to 'singlet'


Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type. The following *XML* snippet needs to be added

In [25]:
readjobfile = """
<readjobfile help="which states to read into the jobfile for each segment type">
     <singlet>Methane:s1</singlet>
     <triplet>Methane:t1</triplet>
     <electron>Methane:e1</electron>
     <hole>Methane:h1</hole>
</readjobfile>
"""

The following code add the `readjobfile` section,

In [26]:
add_section("iqm", readjobfile)

Now, let's write the jobs to the file

In [27]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j "write"

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... iqm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
... ... Writing job file iqm.jobs
... ... In total 12507 jobs
Changes have not been written to state file.


From the jobs that we just write down, let's make available only the first job

In [28]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [29]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j run -c 1 -t 4

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... iqm 
... ... Initialized with 4 threads.

... ... Using 1 openmp threads for 4x1=4 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
MST ERR Job file = 'iqm.jobs', cache size =  1
MST ERR Initialize jobs from iqm.jobs
MST ERR Registered 12507 jobs.
=> [ 0%] 

T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0
T00 ERR ... 2020-5-26 17:38:56 Evaluating pair 0 [0:1] out of 21093
T00 ERR ... Guess requested, reading molecular orbitals
T00 ERR ... Reading Mol

Finally, we read the results into the state

In [30]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -j "read"

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... iqm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iqm 
Writing MD topology (step = 0, time = 0) to state.hdf5
... . 


## Coupling
We can now compute the classical coupling of transition in the aformentioned three stages,

In [31]:
!cp "${VOTCASHARE}/xtp/xml/iexcitoncl.xml" OPTIONFILES

We need to change in the `iexcitoncl` input the name `map_file` option and add the state. check all the available of the [iexcitoncl calculator](https://votca.github.io/iexcitoncl.html).

In [32]:
edit_calculator("iexcitoncl", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/iexcitoncl.xml' has been set to 'system.xml'


In [33]:
state = "<states>Methane:n2s1</states>"
add_section("iexcitoncl", state)

In [34]:
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "write"

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... iexcitoncl 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... iexcitoncl 
... ... Writing job file exciton.jobs
... ... In total 8586 jobs
Changes have not been written to state file.


Now we can run and save the jobs. For demo purposes we will run only the first job

In [35]:
sed -i "s/AVAILABLE/COMPLETE/g" exciton.jobs
sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' exciton.job
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5

In [36]:
#!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "read"

## Coupling analysis
Using the coupling computed in the previous step, we will generate an histogram for the squared couplings in logarithmic scale,

In [37]:
!cp "${VOTCASHARE}/xtp/xml/ianalyze.xml" OPTIONFILES
edit_calculator("ianalyze", "states", "e,h,s")
!xtp_run -e ianalyze -o OPTIONFILES/ianalyze.xml -f state.hdf5

The option 'states' on file 'OPTIONFILES/ianalyze.xml' has been set to 'e,h,s'
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator
... ianalyze 
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... ianalyze 
Calculating for state e now.
Calculating for state h now.
Calculating for state s now.

Changes have not been written to state file.


## QMMM calculations

In [38]:
!cp qmmm.xml OPTIONFILES/                                                                                                                                                                                 
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "write"                                                                                                                                    


 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... qmmm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
... ... Writing job file qmmm_jobs.xml
... ... In total 3001 jobs
Changes have not been written to state file.
 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled Ma

T00 ERR ... 2020-5-26 17:46:11 Evaluating interaction between polar 1 and qm 0
T00 ERR ... 2020-5-26 17:46:12 Evaluating interaction between polar 1 and static 2
T00 ERR ... 2020-5-26 17:46:12 Starting Solving for classical polarization with 1380 degrees of freedom.
T00 ERR ... 2020-5-26 17:46:12 CG: #iterations: 7, estimated error: 5.54889470065e-06
T00 ERR ...   Total static energy [hrt]= 0.0009102654542
T00 ERR ...   Total polar energy [hrt]= -0.0006142561623
T00 ERR ...  Total energy [hrt]= 0.0002960092919
T00 ERR ... 2020-5-26 17:46:12 Evaluating static 2
T00 ERR ... 2020-5-26 17:46:12 Writing checkpoint to checkpoint_iter_1.hdf5
T00 ERR ...  Region:qm 0 is not converged deltaE=-40.47407446 RMS Dmat=0.03339379361 MaxDmat=0.8229286718
T00 ERR ... 2020-5-26 17:46:12 Region:polar 1 is not converged deltaE=0.0002960092919
T00 ERR ... 2020-5-26 17:46:12 --Total Energy all regions -40.47377845
T00 ERR ... 2020-5-26 17:46:12 --Inter Region SCF Iteration 2 of 50
T00 ERR ... 2020-5-26 17:4

T00 ERR ... 2020-5-26 17:46:18 Evaluating interaction between polar 1 and qm 0
T00 ERR ... 2020-5-26 17:46:18 Evaluating interaction between polar 1 and static 2
T00 ERR ... 2020-5-26 17:46:19 Starting Solving for classical polarization with 1380 degrees of freedom.
T00 ERR ... 2020-5-26 17:46:19 CG: #iterations: 0, estimated error: 5.54889470064e-06
T00 ERR ...   Total static energy [hrt]= 0.0009102654542
T00 ERR ...   Total polar energy [hrt]= -0.0006142561623
T00 ERR ...  Total energy [hrt]= 0.0002960092919
T00 ERR ... 2020-5-26 17:46:19 Evaluating static 2
T00 ERR ... 2020-5-26 17:46:19 Writing checkpoint to checkpoint_iter_2.hdf5
T00 ERR ...  Region:qm 0 is converged deltaE=0 RMS Dmat=0 MaxDmat=0
T00 ERR ... 2020-5-26 17:46:19 Region:polar 1 is converged deltaE=0
T00 ERR ... 2020-5-26 17:46:19 --Total Energy all regions -40.47377845
T00 ERR ... 2020-5-26 17:46:19 Job converged after 2 iterations.
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job
T00 ERR ... Assign

Lets run just the first job

In [39]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_jobs.xml                                                                                                                                                            
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml                                                                                                                                                 
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j run

 This is a XTP app
========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_parallel, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:51:24)
votca_csg, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:45:05)
votca_tools, version 1.7-dev gitid: 59c2111 (dirty) (compiled May 26 2020, 10:43:33)

Initializing calculator 
... qmmm 
... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... qmmm 
MST ERR Job file = 'qmmm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_jobs.xml
MST ERR Registered 3000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ...  Regions created
T00 ERR ... Id: 0 type: qm size: 1 charge[e]= -2.80472e+14
T00 ERR ... Id: 1 type: polar size: 92 charge[e]= 

T00 ERR ... 2020-5-26 17:53:44 Evaluating interaction between polar 1 and qm 0
T00 ERR ... 2020-5-26 17:53:45 Evaluating interaction between polar 1 and static 2
T00 ERR ... 2020-5-26 17:53:45 Starting Solving for classical polarization with 1380 degrees of freedom.
T00 ERR ... 2020-5-26 17:53:45 CG: #iterations: 7, estimated error: 5.54889470065e-06
T00 ERR ...   Total static energy [hrt]= 0.0009102654542
T00 ERR ...   Total polar energy [hrt]= -0.0006142561623
T00 ERR ...  Total energy [hrt]= 0.0002960092919
T00 ERR ... 2020-5-26 17:53:45 Evaluating static 2
T00 ERR ... 2020-5-26 17:53:45 Writing checkpoint to checkpoint_iter_1.hdf5
T00 ERR ...  Region:qm 0 is not converged deltaE=-40.47407446 RMS Dmat=0.03339379361 MaxDmat=0.8229286718
T00 ERR ... 2020-5-26 17:53:45 Region:polar 1 is not converged deltaE=0.0002960092919
T00 ERR ... 2020-5-26 17:53:45 --Total Energy all regions -40.47377845
T00 ERR ... 2020-5-26 17:53:45 --Inter Region SCF Iteration 2 of 50
T00 ERR ... 2020-5-26 17:5

T00 ERR ... 2020-5-26 17:53:51 Evaluating interaction between polar 1 and qm 0
T00 ERR ... 2020-5-26 17:53:51 Evaluating interaction between polar 1 and static 2
T00 ERR ... 2020-5-26 17:53:52 Starting Solving for classical polarization with 1380 degrees of freedom.
T00 ERR ... 2020-5-26 17:53:52 CG: #iterations: 0, estimated error: 5.54889470064e-06
T00 ERR ...   Total static energy [hrt]= 0.0009102654542
T00 ERR ...   Total polar energy [hrt]= -0.0006142561623
T00 ERR ...  Total energy [hrt]= 0.0002960092919
T00 ERR ... 2020-5-26 17:53:52 Evaluating static 2
T00 ERR ... 2020-5-26 17:53:52 Writing checkpoint to checkpoint_iter_2.hdf5
T00 ERR ...  Region:qm 0 is converged deltaE=0 RMS Dmat=0 MaxDmat=0
T00 ERR ... 2020-5-26 17:53:52 Region:polar 1 is converged deltaE=0
T00 ERR ... 2020-5-26 17:53:52 --Total Energy all regions -40.47377845
T00 ERR ... 2020-5-26 17:53:52 Job converged after 2 iterations.
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job
T00 ERR ... Assign

Finally, save the results

In [ ]:
#!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "read"